In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet101

In [ ]:
# from unzip import unzip_file

# # 指定zip文件的路径
# zip_file_path = './data/archive.zip'
# extract_to_path = './data/'

# unzip_file(zip_file_path, extract_to_path)

In [19]:
# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
batch_size = 16
num_negative_batch = 16

cuda


In [3]:
# 1. 数据预处理和增强
transform = transforms.Compose([
    transforms.RandomResizedCrop(256),        # 随机裁剪 64x64 图像，因为Tiny-ImageNet图像是64x64
    transforms.RandomHorizontalFlip(p=0.5),   # 50% 概率水平翻转
    transforms.Grayscale(num_output_channels=3), # 转为灰度图
    transforms.ToTensor(),
])

# 2. 加载 Tiny-ImageNet 数据集
train_dir = './data/tiny-imagenet-200/train'
val_dir = './data/tiny-imagenet-200/val'

# 使用 ImageFolder 加载数据集
trainset = ImageFolder(root=train_dir, transform=transform)
valset = ImageFolder(root=val_dir, transform=transform)

# 3. 创建数据加载器
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
valloader = DataLoader(valset, batch_size=batch_size, shuffle=False)

# 查看数据集的一些信息
print(f"训练集大小: {len(trainset)}")
print(f"验证集大小: {len(valset)}")

训练集大小: 100000
验证集大小: 10000


In [21]:
# test
for a,b in trainloader:
    print(a.shape, b.shape, b)
    break

torch.Size([16, 3, 256, 256]) torch.Size([16]) tensor([144,  56, 112, 172, 162,  37, 113, 143, 199, 169, 136,  57, 150, 142,
        111, 159])


In [4]:
# 2. 定义编码器（ResNet-101）并提取第三残差块输出
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.resnet = resnet101(pretrained=False)
        self.resnet = nn.Sequential(*list(self.resnet.children())[:6])  # 提取第三残差块输出

        # 随机裁剪 60x60 并填充回 64x64
        self.random_crop_and_pad = transforms.Compose([
            transforms.RandomCrop(60),    # 随机裁剪出 60x60
            transforms.Pad(2)             # 填充回 64x64（2像素的填充）
        ])

    def forward(self, x):
        # 对每个 256x256 图像进行 7x7 的 64x64 的网格裁剪，每个裁剪之间有32像素的重叠
        patches = x.unfold(2, 64, 32).unfold(3, 64, 32) # 1.在第三个维度上提取大小为64的局部块，步长为32。2.在第四个维度上提取大小为64的局部块，步长为32。
        # print("###", patches.shape, "###") # torch.Size([batch_size, channels, 7, 7, 64, 64])
        patches = patches.contiguous().view(-1, 3, 64, 64)  # 展开为一批 64x64 小块
        # print("###", patches.shape, "###") # torch.Size([784, 3, 64, 64])

        # 对每个 64x64 的小块执行随机 60x60 裁剪并填充回 64x64
        patches = torch.stack([self.random_crop_and_pad(patch) for patch in patches]) # 每一个小patch.shape=torch.Size([3, 64, 64])
        # print("###", patches.shape, "###") # ### torch.Size([784, 3, 64, 64]) ###
        # for patch in patches:
        #     print(self.random_crop_and_pad(patch)) # torch.Size([3, 64, 64])
        
        features = self.resnet(patches)  # 提取特征
        # print("###", features.shape, "###") # ### torch.Size([784, 512, 8, 8]) ###
        features = nn.functional.adaptive_avg_pool2d(features, (1, 1))  # 全局池化
        # print("###", features.shape, "###") # ### torch.Size([784, 512, 1, 1]) ###
        # features = features.view(-1, 512)  # [B, 1024]，每个小块的 1024 维向量
        # print("###", features.shape, "###") # ### torch.Size([392, 1024]) ###
        features = features.view(batch_size, 7, 7, -1)  # 恢复为 [B, 7, 7, 1024]
        # print("###", features.shape, "###") # ### torch.Size([8, 7, 7, 1024]) ###
        # 要把1024调整512才可以适应batch_size

        return features # shape=[batch_size=16, 7, 7, 512]

在你提供的代码中，`features.shape=torch.Size([784, 512, 8, 8])`，表示提取出来的特征张量具有四个维度。我们可以分别解释这些维度：

### 特征张量的维度解释

1. **784 (第一个维度)**：
   - 这是批次中包含的**图像块的数量**。你使用了 `x.unfold(2, 64, 32).unfold(3, 64, 32)` 将输入的图像裁剪成了多个 64x64 的小块（每张 256x256 图像被分成了 7x7 个 64x64 的局部块）。
   - 假设你的批量大小是 1（即一次输入一张 256x256 的图像），那么每张图像会产生 7x7 = 49 个 64x64 的小块。如果有 16 张图像，那么总共有 16 × 49 = 784 个小块，所以这个维度是 **784**。
   - 具体来说：`784 = 16 * 49`，其中 16 是批量大小，49 是每张图像裁剪的小块数量。

2. **512 (第二个维度)**：
   - 这是每个图像块提取出来的**特征通道数**，在你的代码中，这来自 `resnet101` 的第三个残差块输出。`resnet101` 的特征输出在第3个残差块后的通道数是 **512**。
   - 这意味着对于每个 64x64 的图像块，`ResNet` 提取了 512 个特征通道，用于表示图像块的不同特征。

3. **8 (第三个维度)**：
   - 这是提取的特征图的**高度**。由于 `resnet101` 中的卷积和池化操作会逐渐降低特征图的空间维度，输入的 64x64 的图像块经过 ResNet 的前几个残差块后，特征图的空间维度从 64x64 减少到了 8x8。
   - 这个数字表示每个图像块在特征空间中被分成了 8x8 的小块。

4. **8 (第四个维度)**：
   - 这是提取的特征图的**宽度**，同样是由 `resnet101` 的卷积和池化操作导致的。和高度维度类似，原始的 64x64 图像块经过卷积后，宽度也被缩减到了 8。

### 总结：
- **784**：总共有 784 个图像块，这是批量中所有图像被裁剪成的小块总数。
- **512**：每个图像块的特征通道数，即从 `resnet101` 中提取的 512 个通道特征。
- **8x8**：每个图像块在经过 `ResNet101` 后的特征图的空间大小。原始的 64x64 图像块被压缩到 8x8 的特征图。

因此，`features.shape = torch.Size([784, 512, 8, 8])` 表示有 784 个 64x64 的图像块，它们分别被表示为具有 512 个通道和 8x8 空间分辨率的特征图。这些特征可以捕捉到每个块中更高层次的空间和语义信息。

In [ ]:
# test
# if __name__ == "__main__":
#     model = Encoder()
#     x = torch.randn(4, 3, 256, 256)  # 模拟 4 张 256x256 的 RGB 图像
#     output = model(x)
#     print(output.shape)  # 应输出 [4, 7, 7, 1024]，实际输出：torch.Size([2, 7, 7, 1024])

In [10]:
# 3. PixelCNN 风格的自回归模型（使用 GRU 作为示例）
class AutoregressiveModel(nn.Module):
    def __init__(self, input_dim=512, hidden_dim=512):
        super(AutoregressiveModel, self).__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, batch_first=True)

    def forward(self, x):  # shape=[batch_size=16, 7, 7, 512]
        B, H, W, D = x.size()  # [B, 7, 7, 512]
        x = x[:, :2, :, :]  # 取前两行作为 GRU 的输入 [B, 2, 7, 512]
        x = x.view(B, 2 * W, D)  # 展开为序列 [B, 14, 512]

        output, hidden = self.gru(x)  # GRU 输出 [B, 14, hidden_dim]，hidden [1, B, hidden_dim]
        last_hidden = hidden[-1]  # 取最后一个时间步的隐藏状态 [B, hidden_dim]
        # print(last_hidden.shape) # torch.Size([16, 512])

        # 复制 last_hidden 5*7 次，以适配后续的形状 [B, 5, 7, hidden_dim]
        last_hidden = last_hidden.unsqueeze(1).unsqueeze(1)  # [B, 1, 1, hidden_dim]
        last_hidden = last_hidden.expand(B, 5, 7, D)  # [B, 5, 7, 512]

        return last_hidden  # 作为 query 返回

In [ ]:
# test AutoregressiveModel
ARModel = AutoregressiveModel()
test = torch.randn(batch_size, 7, 7, 512)
test = ARModel(test)
# print("###", test)

In [11]:
# 4. CPC模型
class CPCModel(nn.Module):
    def __init__(self):
        super(CPCModel, self).__init__()
        self.encoder = Encoder()
        self.autoregressive = AutoregressiveModel()

    def forward(self, x):
        z = self.encoder(x)  # 提取每个64x64块的特征 [B, 7, 7, 512]
        c = self.autoregressive(z)  # 预测后五行特征 [B, 5, 7, 512]
        return z, c

正样本：是未来时刻的输入（例如7*7的patches，我们选定一个时刻比如第五行最后一列，那么之后的两行都是未来时刻）经过编码器的输出z     
负样本：任意输入，经过编码器的输出都应该和未来时刻的输出经过编码器的输出不相似

问题的关键是：正样本对定义中，自回归模型的输出c和特征提取模型的输出z之间的对应关系

In [ ]:
# 5. 对比损失 (InfoNCE)
# class InfoNCELoss(nn.Module):
#     def __init__(self, temperature=0.07):
#         super(InfoNCELoss, self).__init__()
#         self.temperature = temperature
#         self.criterion = nn.CrossEntropyLoss()

#     def forward(self, z_i, z_j): # criterion(c.reshape(-1, 512), z.reshape(-1, 512)) # z_i，z_j.shape=(16*7*7, 512)
#         B = z_i.size(0) # 16*7*7
#         z_i = nn.functional.normalize(z_i, dim=1) # dim=1代表每行归一化：除以\sqrt{每个元素的平方之和} - 特征归一化
#         z_j = nn.functional.normalize(z_j, dim=1)

#         # 相似性矩阵
#         similarity_matrix = torch.matmul(z_i, z_j.T) / self.temperature # torch.matmul(z_i, z_j.T) - 16*7*7 与 16*7*7 patches 之间的相似度

#         # 标签：对角线位置是正样本
#         labels = torch.arange(B).to(device)
#         loss = self.criterion(similarity_matrix, labels) # similarity_matrix.shape=(16*7*7, 16*7*7), labels.shape=(16*7*7)

#         return loss

In [14]:
# 6. 初始化模型、损失函数和优化器
from InfoNCE import InfoNCE

model = CPCModel().to(device)
criterion = InfoNCE()
optimizer = optim.Adam(model.parameters(), lr=2e-4)

# test loss
batch_size, num_negative, embedding_size = 32, 48, 128
query = torch.randn(batch_size, embedding_size)
positive_key = torch.randn(batch_size, embedding_size)
negative_keys = torch.randn(num_negative, embedding_size)
output = criterion(query, positive_key, negative_keys)
output

tensor(4.2249)

In [22]:
# 7. 训练循环
for epoch in range(10):  # 训练10个epoch
    running_loss = 0.0
    for batch_idx, data in enumerate(trainloader, 0):
        inputs, _ = data
        inputs = inputs.to(device)

        optimizer.zero_grad()

        # 前向传播
        z, c = model(inputs)
        # z.shape = [16, 7, 7, 512]  # ResNet 提取的特征
        # c.shape = [16, 5, 7, 512]  # 自回归模型生成的 query

        # 取 ResNet 提取的后五行特征作为 positive_key
        positive_key = z[:, 2:, :, :].reshape(-1, 512)  # [16*5*7, 512]

        # 将 query 复制并展平 [16*5*7, 512]
        query = c.reshape(-1, 512)

        # 生成随机负样本，并通过 ResNet 编码为负样本特征
        negative_samples = torch.randn(num_negative_batch, 3, 256, 256).to(device)  # 随机采样的负样本
        negative_key = model.encoder(negative_samples)  # 使用 ResNet 编码为特征
        negative_key = negative_key.reshape(-1, 512)  # 展平为 [N_negative, 512]

        # 调用 InfoNCE 损失函数
        loss = criterion(query, positive_key, negative_key)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # 每 100 个 batch 打印一次损失
        if batch_idx % 1 == 0:
            print(f'Batch {batch_idx}/{len(trainloader)}: Loss: {loss.item():.4f}')

print('Finished Training')

D:\Miniconda\envs\DL_learning\lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Batch 0/6250: Loss: 6.6768
Batch 1/6250: Loss: 6.4208
Batch 2/6250: Loss: 6.1362
Batch 3/6250: Loss: 5.8133
Batch 4/6250: Loss: 5.6515
Batch 5/6250: Loss: 5.1535
Batch 6/6250: Loss: 4.9025
Batch 7/6250: Loss: 4.6648
Batch 8/6250: Loss: 4.4817
Batch 9/6250: Loss: 3.9709
Batch 10/6250: Loss: 3.6694
Batch 11/6250: Loss: 3.7614
Batch 12/6250: Loss: 2.9979
Batch 13/6250: Loss: 2.9318
Batch 14/6250: Loss: 3.2087
Batch 15/6250: Loss: 3.1009
Batch 16/6250: Loss: 2.7345
Batch 17/6250: Loss: 2.8084
Batch 18/6250: Loss: 2.0524
Batch 19/6250: Loss: 2.2597
Batch 20/6250: Loss: 2.1454
Batch 21/6250: Loss: 1.6520
Batch 22/6250: Loss: 2.2991
Batch 23/6250: Loss: 2.0737
Batch 24/6250: Loss: 2.1663
Batch 25/6250: Loss: 2.1639
Batch 26/6250: Loss: 1.6485
Batch 27/6250: Loss: 1.1176
Batch 28/6250: Loss: 1.1488
Batch 29/6250: Loss: 1.0839
Batch 30/6250: Loss: 1.3783
Batch 31/6250: Loss: 1.4715
Batch 32/6250: Loss: 0.9963
Batch 33/6250: Loss: 1.4171
Batch 34/6250: Loss: 1.1653
Batch 35/6250: Loss: 1.1420
Ba


KeyboardInterrupt

